In [72]:
import pandas as pd
import nltk
import matplotlib.pyplot as plt
import math

In [17]:
def read_from_file(file_name):
    data = []
    f = open(file_name, mode="r", encoding="utf-8")
    for line in f.readlines():
        data.append({"score":int(line[0:1]), "comment":line[2:]})
    f.close()
    return pd.DataFrame(data)

train = read_from_file("./collectdata/train.txt")
test = read_from_file("./collectdata/valid.txt")

In [47]:
train.groupby("score").count()

,comment
score,
1,45
2,47
3,101
4,349
5,1257


In [102]:
def train_model(**kwargs):
    
    def calc_probabilities_in_class(data):
        bag_of_words = {}
        for index, row in data.iterrows():
            words = nltk.word_tokenize(row['comment'])
            for word in words:
                word_l = word.lower()
                if not word_l in bag_of_words:
                    bag_of_words[word_l] = 1
                else:
                    bag_of_words[word_l] = bag_of_words[word_l] + 1
        return bag_of_words
    
    probabilities = {}
    for key in kwargs:
        probabilities[key] = calc_probabilities_in_class(kwargs[key])
    return probabilities


def predict_comment(model, comment):
    
    def comment_log_probabitlity(words, class_model, total_words):
        denominator = sum(class_model.values()) + len(class_model)
        print("denominator is ", denominator)
        log_prob = math.log(sum(class_model.values()) / total_words)
        print("initial log prob is ", log_prob)
        for word in words:
            n = class_model[word] if word in class_model else 0 
            print("For word ", word,", n of usage is:",n, ", prob:",((n + 1)/denominator))
            log_prob += math.log((n + 1)/denominator)
        return log_prob
        
    log_probablitites = {}
    words = nltk.word_tokenize(comment)
    
    total_words = 0
    for key in model:
        total_words += sum(model[key].values())
        
    for key in model:
        log_probablitites[key] = comment_log_probabitlity(words, model[key], total_words)
    return log_probablitites


def predict_on_set(model, test_set):

    for index, row in test_set.iterrows():
        max_prob = float('-inf')
        max_key = ""
        probs = predict_comment(model, row["comment"])
        print(probs)
        for key_class in probs:
            if probs[key_class] > max_prob:
                max_prob = probs[key_class]
                max_key = key_class
                
        print(max_key, ":", row["score"])
        
    

In [103]:
model_2 = train_model(positive = train[train["score"] > 3], negative = train[train["score"] < 3])
predict_on_set(model, test)

denominator is  42652
initial log prob is  -0.07736008837581661
For word  Купила , n of usage is: 0 , prob: 2.3445559411047548e-05
For word  синові , n of usage is: 0 , prob: 2.3445559411047548e-05
For word  ... , n of usage is: 80 , prob: 0.0018990903122948514
For word  класна , n of usage is: 16 , prob: 0.0003985745099878083
For word  штука..виручає , n of usage is: 0 , prob: 2.3445559411047548e-05
For word  часто , n of usage is: 14 , prob: 0.0003516833911657132
denominator is  4051
initial log prob is  -2.597714984427126
For word  Купила , n of usage is: 0 , prob: 0.00024685262898049864
For word  синові , n of usage is: 0 , prob: 0.00024685262898049864
For word  ... , n of usage is: 13 , prob: 0.003455936805726981
For word  класна , n of usage is: 0 , prob: 0.00024685262898049864
For word  штука..виручає , n of usage is: 0 , prob: 0.00024685262898049864
For word  часто , n of usage is: 0 , prob: 0.00024685262898049864
{'positive': -54.106624059373374, 'negative': -49.79897191402801

For word  мені , n of usage is: 2 , prob: 0.0007405578869414959
For word  повністю , n of usage is: 2 , prob: 0.0007405578869414959
For word  підходить , n of usage is: 0 , prob: 0.00024685262898049864
For word  , , n of usage is: 169 , prob: 0.04196494692668477
For word  плюс , n of usage is: 1 , prob: 0.0004937052579609973
For word  є , n of usage is: 6 , prob: 0.0017279684028634905
For word  запас , n of usage is: 0 , prob: 0.00024685262898049864
For word  для , n of usage is: 10 , prob: 0.0027153789187854853
For word  росту , n of usage is: 0 , prob: 0.00024685262898049864
For word  . , n of usage is: 181 , prob: 0.044927178474450755
{'positive': -595.7516973401941, 'negative': -536.0832590011137}
negative : 5
denominator is  42652
initial log prob is  -0.07736008837581661
For word  Постійно , n of usage is: 0 , prob: 2.3445559411047548e-05
For word  протікає , n of usage is: 8 , prob: 0.00021101003469942793
denominator is  4051
initial log prob is  -2.597714984427126
For word  Пос

For word  ! , n of usage is: 17 , prob: 0.004443347321648976
For word  ! , n of usage is: 17 , prob: 0.004443347321648976
For word  Хороший , n of usage is: 0 , prob: 0.00024685262898049864
For word  вкус , n of usage is: 0 , prob: 0.00024685262898049864
For word  , , n of usage is: 169 , prob: 0.04196494692668477
For word  нет , n of usage is: 0 , prob: 0.00024685262898049864
For word  комков , n of usage is: 0 , prob: 0.00024685262898049864
For word  . , n of usage is: 181 , prob: 0.044927178474450755
{'positive': -69.14372564332373, 'negative': -66.65398501863848}
negative : 5
denominator is  42652
initial log prob is  -0.07736008837581661
For word  доступна , n of usage is: 4 , prob: 0.00011722779705523774
For word  жилка , n of usage is: 0 , prob: 2.3445559411047548e-05
For word  для , n of usage is: 302 , prob: 0.007104004501547407
For word  новачка , n of usage is: 2 , prob: 7.033667823314265e-05
denominator is  4051
initial log prob is  -2.597714984427126
For word  доступна , n

For word  враження , n of usage is: 13 , prob: 0.0003282378317546657
For word  дорогої , n of usage is: 0 , prob: 2.3445559411047548e-05
For word  якісної , n of usage is: 2 , prob: 7.033667823314265e-05
For word  речі , n of usage is: 9 , prob: 0.00023445559411047547
For word  . , n of usage is: 2361 , prob: 0.05537841132889431
For word  Як , n of usage is: 0 , prob: 2.3445559411047548e-05
For word  покаже , n of usage is: 3 , prob: 9.378223764419019e-05
For word  себе , n of usage is: 30 , prob: 0.000726812341742474
For word  в , n of usage is: 494 , prob: 0.011605551908468537
For word  роботі , n of usage is: 6 , prob: 0.00016411891587733284
For word  - , n of usage is: 253 , prob: 0.0059551720904060775
For word  побачимо , n of usage is: 3 , prob: 9.378223764419019e-05
For word  . , n of usage is: 2361 , prob: 0.05537841132889431
denominator is  4051
initial log prob is  -2.597714984427126
For word  Отримав , n of usage is: 0 , prob: 0.00024685262898049864
For word  швидко , n of u

For word  . , n of usage is: 181 , prob: 0.044927178474450755
For word  Якраз , n of usage is: 0 , prob: 0.00024685262898049864
For word  те , n of usage is: 6 , prob: 0.0017279684028634905
For word  що , n of usage is: 35 , prob: 0.008886694643297952
For word  шукав , n of usage is: 0 , prob: 0.00024685262898049864
For word  . , n of usage is: 181 , prob: 0.044927178474450755
{'positive': -233.77618792178333, 'negative': -213.65720508534054}
negative : 5
denominator is  42652
initial log prob is  -0.07736008837581661
For word  Фантастически , n of usage is: 0 , prob: 2.3445559411047548e-05
For word  удобный , n of usage is: 4 , prob: 0.00011722779705523774
For word  , , n of usage is: 2307 , prob: 0.05411235112069774
For word  почти , n of usage is: 2 , prob: 7.033667823314265e-05
For word  невесомый , n of usage is: 0 , prob: 2.3445559411047548e-05
For word  браслет , n of usage is: 26 , prob: 0.0006330301040982837
For word  , , n of usage is: 2307 , prob: 0.05411235112069774
For wor

...

initial log prob is  -0.07736008837581661
For word  Купляла , n of usage is: 0 , prob: 2.3445559411047548e-05
For word  в , n of usage is: 494 , prob: 0.011605551908468537
For word  подарунок , n of usage is: 28 , prob: 0.0006799212229203789
For word  . , n of usage is: 2361 , prob: 0.05537841132889431
For word  Іменинник , n of usage is: 0 , prob: 2.3445559411047548e-05
For word  задоволений , n of usage is: 118 , prob: 0.002790021569914658
For word  . , n of usage is: 2361 , prob: 0.05537841132889431
For word  Все , n of usage is: 0 , prob: 2.3445559411047548e-05
For word  працює , n of usage is: 55 , prob: 0.0013129513270186627
For word  . , n of usage is: 2361 , prob: 0.05537841132889431
denominator is  4051
initial log prob is  -2.597714984427126
For word  Купляла , n of usage is: 0 , prob: 0.00024685262898049864
For word  в , n of usage is: 47 , prob: 0.011848926191063935
For word  подарунок , n of usage is: 3 , prob: 0.0009874105159219946
For word  . , n of usage is: 181 , prob:

For word  ! , n of usage is: 17 , prob: 0.004443347321648976
{'positive': -85.49692830029119, 'negative': -81.32319504217153}
negative : 5
denominator is  42652
initial log prob is  -0.07736008837581661
For word  Нормальні , n of usage is: 0 , prob: 2.3445559411047548e-05
For word  бюджетні , n of usage is: 0 , prob: 2.3445559411047548e-05
For word  вітаміни , n of usage is: 11 , prob: 0.0002813467129325706
For word  . , n of usage is: 2361 , prob: 0.05537841132889431
For word  Гарно , n of usage is: 0 , prob: 2.3445559411047548e-05
For word  приймати , n of usage is: 10 , prob: 0.000257901153521523
For word  зимою , n of usage is: 2 , prob: 7.033667823314265e-05
For word  для , n of usage is: 302 , prob: 0.007104004501547407
For word  підняття , n of usage is: 0 , prob: 2.3445559411047548e-05
For word  імунітету , n of usage is: 0 , prob: 2.3445559411047548e-05
For word  , , n of usage is: 2307 , prob: 0.05411235112069774
For word  чи , n of usage is: 65 , prob: 0.0015474069211291381


For word  грати , n of usage is: 0 , prob: 0.00024685262898049864
For word  . , n of usage is: 181 , prob: 0.044927178474450755
For word  Дитина , n of usage is: 0 , prob: 0.00024685262898049864
For word  в , n of usage is: 47 , prob: 0.011848926191063935
For word  школу , n of usage is: 0 , prob: 0.00024685262898049864
For word  носить , n of usage is: 0 , prob: 0.00024685262898049864
For word  , , n of usage is: 169 , prob: 0.04196494692668477
For word  на , n of usage is: 65 , prob: 0.01629227351271291
For word  перерві , n of usage is: 0 , prob: 0.00024685262898049864
For word  з , n of usage is: 22 , prob: 0.005677610466551469
For word  однокласниками , n of usage is: 0 , prob: 0.00024685262898049864
For word  грають , n of usage is: 0 , prob: 0.00024685262898049864
For word  . , n of usage is: 181 , prob: 0.044927178474450755
For word  Задоволений , n of usage is: 0 , prob: 0.00024685262898049864
For word  . , n of usage is: 181 , prob: 0.044927178474450755
{'positive': -222.4325

For word  з , n of usage is: 22 , prob: 0.005677610466551469
For word  не , n of usage is: 84 , prob: 0.020982473463342383
For word  довірою , n of usage is: 0 , prob: 0.00024685262898049864
For word  до , n of usage is: 13 , prob: 0.003455936805726981
For word  такого , n of usage is: 0 , prob: 0.00024685262898049864
For word  способу , n of usage is: 0 , prob: 0.00024685262898049864
For word  продажу/покупки , n of usage is: 0 , prob: 0.00024685262898049864
For word  товарів , n of usage is: 0 , prob: 0.00024685262898049864
For word  в , n of usage is: 47 , prob: 0.011848926191063935
For word  інтернеті , n of usage is: 0 , prob: 0.00024685262898049864
For word  . , n of usage is: 181 , prob: 0.044927178474450755
For word  Вирішив , n of usage is: 0 , prob: 0.00024685262898049864
For word  замовити , n of usage is: 0 , prob: 0.00024685262898049864
For word  універсальну , n of usage is: 0 , prob: 0.00024685262898049864
For word  мобільну , n of usage is: 0 , prob: 0.00024685262898049

For word  терпимо , n of usage is: 0 , prob: 2.3445559411047548e-05
denominator is  4051
initial log prob is  -2.597714984427126
For word  Не , n of usage is: 0 , prob: 0.00024685262898049864
For word  така , n of usage is: 0 , prob: 0.00024685262898049864
For word  хорошо , n of usage is: 0 , prob: 0.00024685262898049864
For word  камера , n of usage is: 0 , prob: 0.00024685262898049864
For word  як , n of usage is: 18 , prob: 0.004690199950629474
For word  очікувалось , n of usage is: 0 , prob: 0.00024685262898049864
For word  , , n of usage is: 169 , prob: 0.04196494692668477
For word  але , n of usage is: 16 , prob: 0.004196494692668477
For word  терпимо , n of usage is: 0 , prob: 0.00024685262898049864
{'positive': -72.31807771887253, 'negative': -66.44473561297842}
negative : 4
denominator is  42652
initial log prob is  -0.07736008837581661
For word  Просто , n of usage is: 0 , prob: 2.3445559411047548e-05
For word  чудовий , n of usage is: 52 , prob: 0.00124261464878552
For word